# Classificação de sentimentos

Nayanne Janoski e Vicenzo Abichequer

O objetivo do seguinte algoritmo é classificar o sentimento de cada frase utilizada, como treinamento para o mesmo, em "positivo" ou "negativo"

Os dados utilizados foram retirados do site https://inclass.kaggle.com/c/si650winter11.
Ao todo foram utilizadas 6918 frases, sendo todas classificadas como 1 (positivas)
ou 0 (negativas).
Metade destas frases foram utilizadas para o treinamento do algoritmo e a outra metade
para a comparação com os resultados obtidos

Começaremos com as importações dos padrões que iremos utilizar:

In [2]:
import pandas
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB

## Importação dos dados utilizando Pandas

In [3]:
texto = pandas.read_csv('data/training.txt', sep='\t', header=None)
texto.shape

(6918, 2)

In [4]:
texto.sample(5)

,0,1
3908,1,man i loved brokeback mountain!
5219,0,"Is it just me, or does Harry Potter suck?..."
3608,1,man i loved brokeback mountain!
3669,1,dudeee i LOVED brokeback mountain!!!!
3297,1,Brokeback Mountain was an AWESOME movie.


## Definição das porções de teste e treino dos dados 

In [5]:
target = texto[0].values
qData = int(len(target)/4)
targetTrain = np.concatenate((target[:qData], target[-qData:]), axis=0)
targetTest = np.concatenate((target[qData:2*qData], target[-2*qData:-qData]), 
axis=0)

data = texto[1].values
dataTrain = np.concatenate((data[:qData], data[-qData:]), axis=0)
dataTest = np.concatenate((data[qData:2*qData], data[-2*qData:-qData]), axis=0)

## Aplicação do método Naive Bayes

Foi utilizado o método de Naive Bayes para classificação de modelos multinomiais

Este classificador é adequado para a classificação com características discretas,
como por exemplo, contagem de palavras para classificação de textos, aonde o
utilizaremos.

In [6]:
nB_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])

Ajuste do Naive Bayes de acordo com os nossos dados para treinamento

In [7]:
nB_clf.fit(dataTrain, targetTrain)

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

Realização da classificação

In [8]:
predicted = nB_clf.predict(dataTest)
predicted

array([1, 1, 1, ..., 0, 0, 0])

Resultado da classificação

In [9]:
print(np.mean(predicted == targetTest))

0.265471370735


## Utilização do método SVM - SGDClassifier

Este estimador implementa modelos lineares regularizados com aprendizado stochastic gradient descent (SGD), ou seja, o gradiente da perda é estimado a cada amostra de cada vez, e o modelo é atualizado ao longo do caminho com uma taxa de aprendizado.

In [14]:
SVM_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3,
                                           random_state=42))])

Ajuste do SVM de acordo com os nossos dados para treinamento

In [15]:
SVM_clf.fit(dataTest, targetTest) 

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...     penalty='l2', power_t=0.5, random_state=42, shuffle=True, verbose=0,
       warm_start=False))])

Realização da classificação

In [16]:
predicted = SVM_clf.predict(dataTest)

In [17]:
predicted

array([1, 1, 1, ..., 0, 0, 0])

Resultado com o classificador SVM

In [18]:
print(np.mean(predicted == targetTest))

0.997397339503


# Validação Cruzada

## Naive Bayes

In [19]:
scores = cross_val_score(nB_clf, dataTest, targetTest, cv = None, verbose = 3, scoring='accuracy')
np.mean(scores)

[CV]  ................................................................
[CV] ................................. , score=0.749567, total=   0.1s
[CV]  ................................................................
[CV] ................................. , score=0.999132, total=   0.1s
[CV]  ................................................................
[CV] ................................. , score=0.800347, total=   0.1s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s finished


0.84968196370113613

## SVM

In [20]:
scores = cross_val_score(SVM_clf, dataTest, targetTest, cv = None, verbose = 3, scoring='accuracy')
np.mean(scores)

[CV]  ................................................................
[CV] ................................. , score=0.690641, total=   0.1s
[CV]  ................................................................
[CV] ................................. , score=0.992188, total=   0.1s
[CV]  ................................................................
[CV] ................................. , score=0.779514, total=   0.1s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s finished


0.82078087890750362